# SWAG

Will start with the SWAG dataset to get a baseline before moving on to the HellaSwag dataset.

In [1]:
from unittest.mock import patch
from datasets import load_dataset

with patch("datasets.builder.is_remote_filesystem", return_value=False):
    dataset = load_dataset("swag")

In [44]:
from dataclasses import dataclass

NUM_ANSWERS = 4
CHOICES = ["A", "B", "C", "D"]


@dataclass
class Sample:
    """A sample multiple choice question."""

    question: str
    answers: list[str]
    label: int

    @property
    def choices(self) -> list[str]:
        """Return the multiple choice answers."""
        return [ f"{choice}) {option}" for choice, option in zip(CHOICES, self.answers) ]

    @property
    def label_text(self) -> str:
        """Returns the expected correct answer."""
        return self.answers[self.label]

    @property
    def expected(self) -> str:
        """Returns the expected correct answer."""
        return f"{self.expected_choice} {self.label_text}"

    @property
    def expected_choice(self) -> str:
        """Returns the expected correct answer."""
        return f"{CHOICES[self.label]})"



def make_sample(record: dict[str, str]) -> Sample:
    return Sample(record["startphrase"], [record[f"ending{i}"] for i in range(0, NUM_ANSWERS)], label=record["label"])

ds = dataset['train']
it = iter(ds)
make_sample(next(it)), make_sample(next(it)), make_sample(next(it))


(Sample(question='Members of the procession walk down the street holding small horn brass instruments. A drum line', answers=['passes by walking down the street playing their instruments.', 'has heard approaching them.', "arrives and they're outside dancing and asleep.", 'turns the lead singer watches the performance.'], label=0),
 Sample(question='A drum line passes by walking down the street playing their instruments. Members of the procession', answers=['are playing ping pong and celebrating one left each in quick.', 'wait slowly towards the cadets.', 'continues to play as well along the crowd along with the band being interviewed.', 'continue to play marching, interspersed.'], label=3),
 Sample(question='A group of members in green uniforms walks waving flags. Members of the procession', answers=['pay the other coaches to cheer as people this chatter dips in lawn sheets.', 'walk down the street holding small horn brass instruments.', 'is seen in the background.', 'are talking a cou

# Open AI Client

This will open a client library to talk to a local model server hosting the model we wish to eval.

In [10]:
from openai import OpenAI

BASE_URL = "http://llama-cpp-server-cuda.llama:8000/v1"
API_KEY = "sk-xxx"
MODEL_ID = "/data/models/mistral-7b-instruct-v0.1.Q5_K_M.gguf"

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {
            "content": "You are a helpful assistant.",
            "role": "system"
        },
        {
            "content": "What is the capital of France?",
            "role": "user"
        }
    ],
)
for choice in response.choices:
    print(choice.message.content)

 The capital city of France is Paris.


In [32]:
INSTRUCTIONS = "Choose the most plausible continuation for the story. Please answer with the letter of the correct answer."

def multiple_choice_question(sample: Sample) -> str:
    """Make a prompt from a sample."""
    return "\n".join([
        sample.question,
        *sample.choices
    ])

def answer(client: OpenAI, question: str, instructions: str = INSTRUCTIONS) -> str:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {
                "content": instructions,
                "role": "system"
            },
            {
                "content": question,
                "role": "user"
            }
        ],
    )
    choice = response.choices[0]
    return choice.message.content.strip()


sample = make_sample(next(it))
response = answer(client, multiple_choice_question(sample))
sample, response

(Sample(question='Then, the boy left again the court, and then he returns with a racket and a ball. The child serves the ball, and the boy', answers=['applauds at the audience.', 'continue a successful strike grass.', 'has the ball and also lets his falls get the ball.', 'play stiff for stopping and score one shot.'], label=0),
 '<</SYS>> Then, the boy left again the court, and then he returns with a racket and a ball. The child serves the ball, and the boy (D) plays stiff for stopping and scores one shot.')

In [46]:
from tqdm import tqdm

correct = 0
total = 0
it = iter(ds)

In [47]:
from itertools import islice

def batched(iterable, n):
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

N_SHOT = 10
pbar = tqdm(enumerate(batched(it, N_SHOT + 1)))

for i, record_batch in pbar:
    shot_records = record_batch[0:N_SHOT]
    record = record_batch[-1]

    shot_samples = [ make_sample(r) for r in record_batch ]
    shot_questions = [ f"{multiple_choice_question(sample)}\n\nAnswer:\n{sample.expected}\n\n" for sample in shot_samples ]
    instructions = "\n".join([
        INSTRUCTIONS,
        *[
            shot_question
            for shot_question in shot_questions
        ],
    ])

    sample = make_sample(record)
    question = f"{multiple_choice_question(sample)}\n\nAnswer:\n"

    response = answer(client, question, instructions)
    total += 1
    # Sometimes the model may answer something like 'The best answer is A) <answer>'
    # so we'll give it credit.
    if response == sample.expected or sample.expected_choice in response or sample.label_text in response:
        correct += 1
    else:
        print(response)
        print(sample.expected)

    accuracy = (correct / total) * 100
    pbar.set_description(f"{accuracy:2.2f}% - {correct} of {total}")


75.00% - 3 of 4: : 4it [00:51, 12.11s/it] 

D) is open into a web.
B) crashes through the wall of an old ruin.


60.00% - 3 of 5: : 5it [01:04, 12.33s/it]

C) is seen running down the land.
A) is wearing black shorts and a shirt.


72.73% - 8 of 11: : 11it [03:04, 16.07s/it]

...

Answer: D) gets up as he reaches a guy across behind her.
C) gives a thumbs up.


66.27% - 167 of 252: : 252it [53:23, 12.34s/it]

B) is interview to the camera while still jogging along the yard.
A) is seen speaking to the camera and continues to run around the beach area.


66.80% - 171 of 256: : 256it [54:08, 12.69s/it]


KeyboardInterrupt: 